In [1]:
# df = num_rowd.copy()
# df['year'] = df.recordtime.dt.year
# df['quarter'] = df.recordtime.dt.to_period('Q').dt.strftime('%q').astype(int)
# df['monthly'] = df.recordtime.dt.strftime('%B')
# df['weekly'] = df.recordtime.dt.strftime('%W').astype(int)
# df['daily'] = df.recordtime.dt.date

# df.head(10)

In [27]:
import numpy as np
import pandas as pd
import dataset as db

from datetime import datetime

In [3]:
num_rowd = db.dbNumCrowd()

In [4]:
def filter_data(data, store, date = None, year = None, week = None, month = None, quarter = None):
    if date:
        data = data[data.recordtime.dt.date == date]
    else:
        data = data[data.recordtime.dt.year == year]
        if isinstance(week, int):
            data = data[data.recordtime.dt.strftime('%W').astype(int) == week]
        elif month:
            data = data[data.recordtime.dt.strftime('%B') == month]
        elif isinstance(quarter, int):
            data = data[data.recordtime.dt.to_period('Q').dt.strftime('%q').astype(int) == quarter + 1]
    
    if store > 0:
        data = data[data.storeid == store]

    return data.sort_values(by = 'recordtime', ascending = True).reset_index(drop = True)

In [5]:
def set_variables(df, store_selected = 0, option_selected = 'Daily'):
    data = df.copy()
    date_selected, week_selected, month_selected, quarter_selected, year_selected = None, None, None, None, None

    if option_selected == 'Daily':
        date_selected = datetime(2022, 6, 2).date()
    else:
        year_selected = 2023
        if option_selected == 'Weekly':
            week_selected = 2
        elif option_selected == 'Monthly':
            month_selected = 'January'
        elif option_selected == 'Quarter':
            quarter_selected = 0

    return data, store_selected, date_selected, week_selected, month_selected, quarter_selected, year_selected


In [6]:
def clean_data(data, option, period = None):
    df = data.copy()

    df.drop(columns = ['position', 'storeid'], axis = 1, inplace = True)

    df['in_num'] = df.in_num.where(df.in_num < 200, df.in_num * 0.01).apply(np.int64)
    df['out_num'] = df.out_num.where(df.out_num < 200, df.out_num * 0.01).apply(np.int64)

    if option == 'Daily':
        freqs = ['5min', '15min', '30min', 'H']
        df = df.resample(freqs[period], on = 'recordtime').sum()
    elif option == 'Weekly':
        df = df.resample('D', on = 'recordtime').sum().reset_index()
        df['day_name'] = df.recordtime.dt.day_name()
        df.set_index(['recordtime', 'day_name'], inplace = True)
    elif option == 'Monthly':
        df = df.resample('D', on = 'recordtime').sum()#.reset_index()
    else:
        df = df.resample('M', on = 'recordtime').sum().reset_index()
        df['recordtime'] = df.recordtime.dt.strftime('%Y-%m')
        df.set_index('recordtime', inplace = True)
    
    return df

In [7]:
option = ['Daily', 'Weekly', 'Monthly', 'Quarter', 'Yearly']
option_selected = option[1]

variables = set_variables(num_rowd.copy(), option_selected = option_selected)

data = variables[0]
store_selected = variables[1]
date_selected = variables[2]
week_selected = variables[3]
month_selected = variables[4]
quarter_selected = variables[5]
year_selected = variables[6]

df = filter_data(data, store_selected, date_selected, year_selected, week_selected, month_selected, quarter_selected)

In [8]:
period_selected = 3

df_1 = clean_data(df, option_selected, period_selected)
df_1

,,in_num,out_num
recordtime,day_name,,
2023-01-09,Monday,15964,16395
2023-01-10,Tuesday,17535,18342
2023-01-11,Wednesday,17664,17799
2023-01-12,Thursday,16855,16858
2023-01-13,Friday,18527,18410
2023-01-14,Saturday,23940,23954
2023-01-15,Sunday,23592,24163


In [32]:
status = db.dbStatus()

status.sort_values('storeid', ascending = True).head(5)

,ID,storeid,FlashNum,RamNum,RC1,RC2,RC3,RC4,RC5,RC6,RC7,RC8,DcID,FV,DcTime,DeviceID,IA,OA,S,T
0,1,13,118,1234,True,False,False,False,False,False,True,False,17,5122,2015-12-29 13:34:00,273,999999,999999,55,2015-12-29 13:34:00
1,2,14,0,12,False,True,False,False,False,False,False,False,18,5122,2015-12-29 13:35:00,283,999999,999999,0,2015-12-29 13:34:00
5,122525,28,6001,0,False,True,False,False,False,False,False,False,28,5896,2023-04-07 06:00:08,450,656659,753448,0,2023-04-06 23:36:00
7,43222,30,1463,0,False,False,False,False,False,False,False,True,17,5896,2019-05-25 03:00:05,280,47466,48671,8,2019-05-24 23:47:00
6,1759,31,0,1,False,False,False,False,True,False,False,False,17,5896,2018-08-30 11:00:09,277,53847,54239,0,2018-08-30 10:58:00


In [75]:
stores = db.dbStore()
err_log = db.dbErrLog()

# err_log.sort_values('LogTime', ascending = False)

In [88]:
stores[['tid', 'name']]

,tid,name
0,28,Floor 02 ...
1,30,Floor 03 ...
2,31,Floor 04 ...
3,32,Floor 05 ...
4,33,Floor 1A (Highlands Coffee) ...
5,34,Floor 1B (Starbucks) ...
6,35,Floor 1P ...
7,36,Floor B ...
8,37,Floor MB ...


In [107]:
g_err_log = err_log.groupby(['storeid', 'Errorcode', 'ErrorMessage']).max()

g_err_log = g_err_log.drop(columns = ['ID', 'DeviceCode'], axis = 1).reset_index()
# g_err_log.reset_index(inplace = True)
g_err_log = g_err_log.merge(stores[['tid', 'name']].rename(columns = {'tid': 'storeid'}),
                            on = 'storeid', how = 'left').set_index('LogTime')#, inplace = True)

g_err_log
# a

,storeid,Errorcode,ErrorMessage,name
LogTime,,,,
2023-03-22 18:34:00,28,1,"No Counting，RX Infrared blocked off,Pls Check ...",Floor 02 ...
2023-04-10 19:51:00,28,8,Counter is dormant ...,Floor 02 ...
2023-02-23 06:26:38,28,22,RX 02 wireless signal lost! Pls Check RX02 sta...,Floor 02 ...
2023-03-22 18:15:57,28,31,"Offline data read error, please Reset the data...",Floor 02 ...
2019-05-25 14:07:00,30,1,"No Counting，RX Infrared blocked off,Pls Check ...",Floor 03 ...
2019-04-26 10:46:00,30,4,Counter is Under Low Battery ...,Floor 03 ...
2019-04-21 19:58:00,30,5,RX Infrared blocked off and Under Low Battery ...,Floor 03 ...
2019-05-24 12:04:00,30,8,Counter is dormant ...,Floor 03 ...
2019-04-23 12:13:00,30,9,"未知的错误码：9,请在config.toml里进行配置 ...",Floor 03 ...
